In [1]:
import pandas as pd
import numpy as np

# load data into dataframe
gold = pd.read_csv("../Data/Gold-1H.csv", sep=',')

# show head of dataframe
gold.head()

,timestamp,price
0,01/01/2010 00:00,1097.88
1,01/01/2010 01:00,1097.88
2,01/01/2010 02:00,1097.88
3,01/01/2010 03:00,1097.88
4,01/01/2010 04:00,1097.88


In [2]:
from rsi_strategy import RsiStrategy

# calculate RSI indicator and add to dataframe
gold['rsi'] = RsiStrategy.calc_rsi(gold['price'], 14)

# show tail of dataframe 
gold.tail()

,timestamp,price,rsi
68348,01/12/2020 20:00,1813.658,84.689963
68349,01/12/2020 21:00,1814.484,87.332911
68350,01/12/2020 22:00,1815.660,85.965580
68351,01/12/2020 23:00,1815.660,84.737077
68352,02/12/2020 00:00,1811.390,73.717019


In [3]:
# calculate changes in rsi between each hour
gold['rsi_delta'] = gold['rsi'] - gold['rsi'].shift(1)

# show latest trades made
gold.tail()

,timestamp,price,rsi,rsi_delta
68348,01/12/2020 20:00,1813.658,84.689963,-2.814399
68349,01/12/2020 21:00,1814.484,87.332911,2.642947
68350,01/12/2020 22:00,1815.660,85.965580,-1.367331
68351,01/12/2020 23:00,1815.660,84.737077,-1.228503
68352,02/12/2020 00:00,1811.390,73.717019,-11.020057


In [4]:
# remove first 30 rows (first rows usually quite noisy)
gold = gold.iloc[30:]

# calculate trades based on rsi and price data
df = pd.DataFrame(RsiStrategy.calc_trades(gold['timestamp'], gold['price'], gold['rsi']))

# add names to columns 
df.columns = ['timestamp', 'entry', 'position_type', 'exit']

# show latest trades made
df.tail()

,timestamp,entry,position_type,exit
2544,23/11/2020 16:00,1834.908,long,1812.912
2545,24/11/2020 12:00,1810.393,long,1816.178
2546,26/11/2020 09:00,1816.178,short,1778.632
2547,27/11/2020 14:00,1778.632,long,1787.252
2548,01/12/2020 06:00,1787.252,short,1806.661


In [5]:
# intersect two dataframes 
df = pd.merge(df, gold, how='inner', on=['timestamp'])

# remove duplicate columns
df.drop(columns=['price'])

# show head 
df.head()

,timestamp,entry,position_type,exit,price,rsi,rsi_delta
0,04/01/2010 06:00,1103.020,short,1116.310,1103.020,73.028674,2.683846
1,04/01/2010 13:00,1114.770,short,1126.795,1114.770,80.693247,-4.080338
2,05/01/2010 08:00,1127.685,short,1139.615,1127.685,85.138451,7.597975
3,06/01/2010 21:00,1137.825,short,1129.835,1137.825,71.933835,-0.723051
4,07/01/2010 10:00,1129.835,long,1154.100,1129.835,18.833652,-17.406372


In [6]:
# load data into dataframes
US10Y = pd.read_csv("../Data/us10y.csv", sep=',')
vix = pd.read_csv("../Data/vix.csv", sep=',')
dollar_index = pd.read_csv("../Data/dxy.csv", sep=',')

# calculate rsi for DXY and remove everything else 
dollar_index['dxy_rsi'] = RsiStrategy.calc_rsi(dollar_index['Close'], 14)
dollar_index = dollar_index.drop(['Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close'], axis=1)

# calculate rsi for US10Y and remove everything else 
US10Y['us10y_rsi'] = RsiStrategy.calc_rsi(US10Y['Close'], 14)
US10Y = US10Y.drop(['Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close'], axis=1)

# remove everything else for the vix except from the close 
# N.B. refer to report why rsi is not calculated for vix 
vix = vix.drop(['Open', 'High', 'Low', 'Volume', 'Adj Close'], axis=1)
vix = vix.rename(columns={"Close": "vix_close"})

# convert Date columns to DateTime objects
US10Y['Date'] = pd.to_datetime(US10Y["Date"])
vix['Date'] = pd.to_datetime(vix["Date"])
dollar_index['Date'] = pd.to_datetime(dollar_index["Date"])

# combine data into one dataframe
fundamental_df = pd.merge(dollar_index, vix, how='outer', on=['Date'])
fundamental_df = pd.merge(fundamental_df, US10Y, how='outer', on=['Date'])

# forward fill NaN values
fundamental_df = fundamental_df.fillna(method='ffill')

# drop starting NaN values that will not be forward filled 
fundamental_df = fundamental_df.dropna()

# show head of dataframe
fundamental_df.head()

,Date,dxy_rsi,vix_close,us10y_rsi
49,2010-03-18,39.285740,16.620001,58.119658
50,2010-03-19,54.819297,16.969999,58.823529
51,2010-03-21,45.255550,16.969999,63.926941
52,2010-03-22,47.509629,16.870001,44.755245
53,2010-03-23,52.097960,16.350000,50.625000


In [7]:
# rename column to match timestamp column name in df Dataframe 
fundamental_df =  fundamental_df.rename(columns={"Date": "timestamp"})

# convert date to string 
fundamental_df['timestamp'] = fundamental_df['timestamp'].dt.strftime('%d/%m/%Y')

# repeat time column 24 times and restore column names
f_df = pd.DataFrame(np.repeat(fundamental_df.values,24,axis=0)) 
f_df.columns = fundamental_df.columns

# list comprehension to brute force all possible times with dates 
new_timelist = [x + " " + str(counter % 24) + ":00" if counter % 24 >= 10 else x + " 0" + str(counter % 24) + ":00" for x, counter in zip(f_df['timestamp'], range(0,len(f_df['timestamp'])))]

# set this as the new time column in f_df
f_df['timestamp'] = new_timelist

# attempt merge 
df = pd.merge(df, f_df, how='inner', on=['timestamp'])

# show result 
df.tail()

,timestamp,entry,position_type,exit,price,rsi,rsi_delta,dxy_rsi,vix_close,us10y_rsi
2428,23/11/2020 11:00,1865.664,long,1841.614,1865.664,28.276438,-5.165120,42.2679,22.66,15.534
2429,23/11/2020 16:00,1834.908,long,1812.912,1834.908,9.795285,-1.581917,42.2679,22.66,15.534
2430,24/11/2020 12:00,1810.393,long,1816.178,1810.393,18.110218,-4.898590,32.7999,21.64,24.6753
2431,27/11/2020 14:00,1778.632,long,1787.252,1778.632,16.482364,-22.680582,26.5305,20.84,19.457
2432,01/12/2020 06:00,1787.252,short,1806.661,1787.252,71.834762,12.636054,6.84915,20.77,57.8704


In [8]:
# create outcome column for prediction
df['outcome'] = df['exit'] - df['entry']
df['outcome'] = (df['outcome'] > 0) & (df['position_type'] == 'long') | (df['outcome'] < 0) & (df['position_type'] == 'short')

# drop entry exit and price 
df = df.drop(columns=['entry', 'exit', 'price'], axis=1, inplace=False)

# show head 
df.tail()

,timestamp,position_type,rsi,rsi_delta,dxy_rsi,vix_close,us10y_rsi,outcome
2428,23/11/2020 11:00,long,28.276438,-5.165120,42.2679,22.66,15.534,False
2429,23/11/2020 16:00,long,9.795285,-1.581917,42.2679,22.66,15.534,False
2430,24/11/2020 12:00,long,18.110218,-4.898590,32.7999,21.64,24.6753,True
2431,27/11/2020 14:00,long,16.482364,-22.680582,26.5305,20.84,19.457,True
2432,01/12/2020 06:00,short,71.834762,12.636054,6.84915,20.77,57.8704,False


In [9]:
# export dataframe to csv 
df.to_csv('training_data.csv')